<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/doublelow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as df
import numpy as np
import scipy.stats as stat
import statsmodels.api as sm

# mbp<65, bis<20, mortality
df = df.read_csv('https://raw.githubusercontent.com/vitaldb/examples/master/doublelow.csv')
df['death'] = df['death'].astype(bool)

# summary
print(df.groupby('death').mean())
print()

# 각 변수 별로
for c in ['mbp_under_65', 'bis_under_20', 'double_low']:
    a = df[df['death']][c]
    b = df[~df['death']][c]

    # group comparison
    print('Group Comparison ({})'.format(c))
    equal_var = stat.levene(a, b)[1] > 0.05  # levene의 등분산 검정
    pval = stat.ttest_ind(a, b, equal_var=equal_var)[1]
    print('T-test p-value={}'.format(pval))
    print()

    # univariate logistic regression
    print('Univariate Logistic Regression ({})'.format(c))
    df['intercept'] = 1
    model = sm.Logit(df['death'], df[['intercept', c]])
    res = model.fit()
    b = res.params[c]
    pval = res.pvalues[c]
    print('{}\tb={:.3f}, exp(b)={:.3f}, pval={:.3f}'.format(c, b, np.exp(b), pval))
    print()


       mbp_under_65  bis_under_20  double_low
death                                        
False     14.180582      0.393411    0.104326
True      22.619255      2.094643    0.421868

Group Comparison (mbp_under_65)
T-test p-value=0.008757785748463904
Univariate Logistic Regression (mbp_under_65)
Optimization terminated successfully.
         Current function value: 0.490814
         Iterations 5
mbp_under_65	b=0.015, exp(b)=1.015, pval=0.006

Group Comparison (bis_under_20)
T-test p-value=0.22635645589599399
Univariate Logistic Regression (bis_under_20)
Optimization terminated successfully.
         Current function value: 0.494882
         Iterations 6
bis_under_20	b=0.064, exp(b)=1.066, pval=0.193

Group Comparison (double_low)
T-test p-value=0.07362792567203962
Univariate Logistic Regression (double_low)
Optimization terminated successfully.
         Current function value: 0.497650
         Iterations 5
double_low	b=0.132, exp(b)=1.142, pval=0.123

